In [1]:
import pandas as pd

df=pd.read_csv('train.csv')

#df.plot.hist()
labels= df.Label
#print(labels)

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import plot_model, to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

import glob
import imageio
import numpy as np
import os.path as path
from datetime import datetime

# Image processing
IMAGE_PATH='train_images'

image_input_files = sorted(glob.glob(path.join(IMAGE_PATH, '*.png')))
images = [imageio.imread(im) for im in image_input_files]

Using TensorFlow backend.


In [3]:
images = np.asarray(images)
#images = images[..., None]
n_images = len(image_input_files)

print(n_images)

#images = images.reshape(,180,1500,3)
image_size = np.asarray([images.shape[0], images.shape[1], images.shape[2]])
print(image_size)
#images = np.expand_dims(images,axis=0)
#print(image_size)
images = images/256

2528
[2528  128  128]


In [5]:
from sklearn.model_selection import train_test_split

num_classes = 6
img_rows = 128
img_cols = 128

# Training set processing
# Split into test and training sets
#TRAIN_TEST_SPLIT = 0.3

# Split at the given index
#split_index = int(TRAIN_TEST_SPLIT * n_images)
#shuffled_indices = np.random.permutation(n_images)
#train_indices = shuffled_indices[0:split_index]
#test_indices = shuffled_indices[split_index:]

#print("split_index=", split_index)
#print("shuffled_indices=",shuffled_indices)
#print("train_indices", train_indices)
#print("test_indices=",test_indices)

# Split the images and the labels
#x_train = images[train_indices, :, :]
#y_train = labels[train_indices]
    
#for i in range(1, n_images):
#     labels[i] = int(i / 3)

#labels=array(labels)

#print(labels_encoded)

#train_indices=n_images-1
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.30)

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
    
y_train=to_categorical(y_train, num_classes)
y_test=to_categorical(y_test, num_classes)

print(y_train)
#---------------------------------------
# CNN network processing
model = Sequential()

# 1st layer
model.add(Conv2D(filters = 64, kernel_size = (5,5), padding = 'same', input_shape = (128,128,1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

#2nd layer
model.add(Conv2D(64, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

#3rd layer
model.add(Conv2D(32, kernel_size = (3,3), padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()
#plot_model(model, to_file='model.png', show_shapes=True)
#optim = SGD(lr=10e-3,momentum=0.0, decay=0.0, nesterov=False)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

BATCH_SIZE=100
EPOCHS= 50

# Train the model
history=model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)


[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 ...
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 128, 128, 64)      1664      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 42, 42, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 42, 42, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 42, 42, 64)        36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 14, 14, 64)        0         
__________

In [6]:
epoch_now=50

In [10]:
history=model.fit(x_train, y_train, epochs=10, batch_size=BATCH_SIZE, verbose=1, initial_epoch=epoch_now)
epoch_now=epoch_now+10

In [11]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


759/759 [==============================] - 6s 8ms/step
Test loss: 0.22462791876869867
Test accuracy: 0.924901185770751


In [12]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

dict_keys([])


NameError: name 'plt' is not defined

In [ ]:
model.save("aoi.h5")